In [8]:
from keras.layers import Input, LSTM, Lambda, Dense,Reshape, Embedding
from keras import backend as K
from keras.models import Model
import numpy as np
from nltk.tokenize import word_tokenize
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [15]:
# Define the input layers for the two strings
string1 = "Read email"
string2 = "Read mail"
similarity_score = 1

# Tokenize the strings
tokenizer = Tokenizer()
tokenizer.fit_on_texts([string1, string2])

# Convert the strings to sequences of integers
string1_seq = tokenizer.texts_to_sequences([string1])[0]
string2_seq = tokenizer.texts_to_sequences([string2])[0]

max_length = max(len(string1_seq), len(string2_seq))

# Pad the sequences to the same length
string1_seq = pad_sequences([string1_seq], maxlen=max_length, padding='post')
string2_seq = pad_sequences([string2_seq], maxlen=max_length, padding='post')


In [16]:
# Define the input layers for the two texts
text1_input = Input(shape=(max_length,))
text2_input = Input(shape=(max_length,))
embedding_layer = Embedding(4, 100, input_length=50)


In [17]:
encoded_text1 = embedding_layer(text1_input)
encoded_text2 = embedding_layer(text2_input)

# Define the LSTM layer to process the texts
shared_lstm = LSTM(64)

# Process the two texts with the shared LSTM layer
encoded_text1 = shared_lstm(encoded_text1)
encoded_text2 = shared_lstm(encoded_text2)

In [18]:
# Define the Lambda layer to calculate the distance between the outputs
distance = Lambda(lambda x:  K.abs(x[0] - x[1]))([encoded_text1, encoded_text2])


In [21]:
# Define the output layer
output = Dense(1, activation='sigmoid')(distance)

# Create the Siamese model
model = Model([text1_input, text2_input], output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')
# Fit the model on the tokenized sequences
model.fit( [string1_seq, string2_seq], np.array([similarity_score]), epochs=10)

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 0.6826
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6746
Epoch 3/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6666
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6586
Epoch 5/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6506
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6426
Epoch 7/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6348
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6273
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6196
Epoch 10/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6120


In [35]:
# Define a new pair of strings
string1 = "Read email"
string2 = "Read mail"

# Tokenize the new strings and pad them to the same length
new_string1_seq = tokenizer.texts_to_sequences([new_string1])[0]
new_string1_seq = pad_sequences([new_string1_seq], maxlen=max_length, padding='post')
new_string2_seq = tokenizer.texts_to_sequences([new_string2])[0]
new_string2_seq = pad_sequences([new_string2_seq], maxlen=max_length, padding='post')

# Use the model to predict the similarity score for the new pair of strings
similarity_score = model.predict([new_string1_seq, new_string2_seq])[0]
print("Similarity Score: ", similarity_score)

1/1 [==============================] - 0s 22ms/step
Similarity Score:  [0.5024927]
